# Scraping from Vivino w/ Selenium

### Importing libraries

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

import time

### Loading dataset

In [2]:
wines = pd.read_csv("../data/wines_clean.csv")

In [3]:
wines.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
wines["ID"] = wines.index + 1

In [5]:
wines.columns

Index(['Name', 'Country', 'Region', 'Winery', 'Rating', 'NumberOfRatings',
       'Price', 'Year', 'WineStyle', 'Full_Name', 'ID'],
      dtype='object')

In [6]:
wines.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year,WineStyle,Full_Name,ID
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011,red,Pomerol 2011 Château La Providence red,1
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017,red,Lirac 2017 Château Mont-Redon red,2
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015,red,Erta e China Rosso di Toscana 2015 Renzo Masi red,3
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019,red,Bardolino 2019 Cavalchina red,4
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016,red,Ried Scheibner Pinot Noir 2016 Markowitsch red,5


###  FOR loop

In [7]:
PATH = "/Users/Angela/Desktop/chromedriver"

In [13]:
# start
driver = webdriver.Chrome(PATH)
driver.set_window_size(1880, 900)

#### Function to scroll down and get to the ``Reviews``section

In [14]:
def scroll_review(driver):
    try:
        driver.execute_script("window.scroll(0, 1000);")
        driver.find_element_by_xpath('//*[@id="all_reviews"]/h2')
    except:
        scroll_review(driver)

#### First let's iterate through with the first 100 bottles

In [15]:
intento = wines[:100]

In [16]:
bottles = []

In [ ]:
iteracion = 0

for i in intento["Full_Name"]:
    iteracion += 1
    
    bottle_info = {}
    bottle_info["ID"] = intento["ID"][iteracion]
    
    x = i.replace(" ", "+").replace("â","a")
    link_find = f"https://www.vivino.com/search/wines?q={x}"
    driver.get(link_find)
    
    #time.sleep(3)
    
    WebDriverWait(driver=driver, timeout=5).until(
    expected_conditions.presence_of_element_located((By.XPATH,
        '/html/body/div[3]/section[1]/div/div/div/div[1]/div/div[1]/div/div[1]'
               ))
    )
    
    driver.execute_script("window.stop();")
    
    try:
        driver.find_element_by_xpath('/html/body/div[3]/section[1]/div/div/div/div[1]/div/div[1]/div/div[1]').click()
        #time.sleep(3)
        
        WebDriverWait(driver=driver, timeout=3).until(
            expected_conditions.presence_of_element_located((By.XPATH,
                '//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody/tr[2]/td'
                   ))
        )
        
        try:
            bottle_info["rating"] = driver.find_element_by_xpath('//*[@id="wine-location-header"]/div[2]/div[1]/a/div/div[1]').text
        except: 
            bottle_info["rating"] = "unknown"
        
        scroll_review(driver)
        
        #time.sleep(5)
        
        WebDriverWait(driver=driver, timeout=5).until(
            expected_conditions.presence_of_element_located((By.XPATH,
                '//*[@id="all_reviews"]/div[2]/div[1]/div[1]/div/div[1]/a/span[2]'
                   ))
        )
        
        try:
            bottle_info["review"] = driver.find_element_by_xpath('//*[@id="all_reviews"]/div[2]/div[1]/div[1]/div/div[1]/a/span[2]').text
        except:
            bottle_info["review"] = "unknown"
        
        bottles.append(bottle_info)
    
    except:
        print(link_find)

print(bottles)

In [ ]:
bottles

---

## Appendix

####  We want to extract the following the information to enrich the dataset:
- Rating
- Price
- Facts x2
- Grapes
- Taste
- Food
- Reviews

### Rating

In [ ]:
rating = driver.find_element_by_xpath('//*[@id="wine-location-header"]/div[2]/div[1]').text
print({"Rating":rating})

### Price

In [ ]:
price = driver.find_element_by_xpath('//*[@id="purchase-availability"]/div/div/div[1]/div[1]').text
print({"Price":price})

### Fact 1

In [ ]:
fact1 = driver.find_element_by_xpath("/html/body/div[2]/div[5]/div/div/div[5]/div[1]/div").text
print({"Fact 1":fact1})

### Fact 2

In [ ]:
fact2 = driver.find_element_by_xpath("/html/body/div[2]/div[5]/div/div/div[5]/div[2]/div").text
print({"Fact 2":fact2})

### Grapes

In [ ]:
grapes = driver.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody/tr[2]/td').text
print({"Grapes":grapes})

### Taste attributes

In [ ]:
att = driver.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div/div[1]/div/div[4]/div/div[2]/div[1]/table/tbody/tr[1]/td[2]/div/span').get_attribute("style")
att = att.split(";")
att = att[1]
print({"Attributes":att})

### Taste description

In [ ]:
taste_desc = driver.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div/div[1]/div/div[4]/div/a[1]/div[2]/div[1]').text
print({"Taste":taste_desc})

### Food

In [ ]:
food = driver.find_element_by_xpath('//*[@id="wine-page-lower-section"]/div/div[1]/div/div[6]/div/div[2]/div').text

print({"Food":food})

### Review

In [ ]:
review = driver.find_element_by_xpath('//*[@id="all_reviews"]/div[3]/div[1]/div[1]/div[1]/div[1]/a/span[3]').text
print({"Review":review})

### NOTE:
Due to lack of resources and time, this idea for my final project hasn't been possible to execute. The kernel would "die" when I tried to iterate through the dataset due to the high amount of data, and because Vivino temporarily blocked my IP Address for trying to extract so much information. Having said this, this idea remains pending for the future.